In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import read_parquet
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle

In [5]:
df_street_busyness=pd.read_parquet('street_zones_factor.parquet')

In [3]:
unique_values=[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 
               140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 
               231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

In [4]:
# set an list for saving predict results
predictions = []
# input of the model, here just a sample
X_input = pd.DataFrame(np.array([[-11.2, 1, 2]]), columns=['temp', 'month', 'time_of_week'])

# loop all model 
for zone_id in unique_values:
    # load model
    filename = 'model-' + str(zone_id) + '.pkl'
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    
    # predict
    prediction = model.predict(X_input)[0]
    # save reslut and zone ID into dictionary--prediction
    predictions.append({'Taxi_Zone_ID': zone_id, 'calm rate': prediction})

# transform dictionary into dataframe
df_predictions = pd.DataFrame(predictions)

In [ ]:
# merge results with streets busyness
df_merged = pd.merge(df_predictions, df_street_busyness, how='left',on='Taxi_Zone_ID')

In [ ]:
# save
df_merged.to_parquet('predictions.parquet', index=False)